In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('C:/workspace/data/alice/labeledTrainData.tsv',sep='\t',quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [7]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />',' ')

In [8]:
# 구둣점, 숫자 제거 -> 영어 이외의 문자는 공백으로 변환
import re

df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]',' ',x))

In [9]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

# 트레인/태스트 데이터 셋 분리 및 텍스트 변환

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(
    df.review,df.sentiment,stratify=df.sentiment, random_state=2021
)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18750,), (6250,), (18750,), (6250,))

## - CountVectorizer

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english',ngram_range=(1,2))
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [14]:
X_train.shape, X_test.shape

((18750,), (6250,))

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train_cv, y_train)

KNeighborsClassifier()

In [18]:
from sklearn.metrics import accuracy_score
pred = knn.predict(X_test_cv)
accuracy_score(y_test, pred)

0.54736

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

C:\Users\CPB06GameN\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
from sklearn.metrics import accuracy_score
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.88656

## - TfidfVectorizer

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
tvect.fit(X_train)
X_train_tf = tvect.transform(X_train)
X_test_tf = tvect.transform(X_test)

In [25]:
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_tf, y_train)
pred_tf = lr.predict(X_test_tf)
accuracy_score(y_test, pred_tf)

0.87904

# 모델 저장하고 불러오기

In [26]:
import joblib

In [29]:
joblib.dump(tvect,'model/imdb_tvect.pkl')
joblib.dump(lr, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [30]:
del tvect
del lr

In [31]:
new_tvect = joblib.load('model/imdb_tvect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [32]:
new_X_test_tf = new_tvect.transform(X_test)
new_pred = new_lr.predict(new_X_test_tf)
accuracy_score(y_test, new_pred)

0.87904

# pipeline과 GridSearchCV를 통한 하이퍼 파라메터 튜닝

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(stop_words='english',ngram_range=(1,2))),
    ('lr',LogisticRegression())
])
params = {
    'tvect__max_df': [100,500],
    'lr__C':[1,10]
}

In [38]:
from  sklearn.model_selection import GridSearchCV
pipe_grid = GridSearchCV(
    pipeline, param_grid=params, cv=3,
    scoring='accuracy', verbose=1,n_jobs=1
)
pipe_grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr', LogisticRegression())]),
             n_jobs=1,
             param_grid={'lr__C': [1, 10], 'tvect__max_df': [100, 500]},
             scoring='accuracy', verbose=1)

In [39]:
pipe_grid.best_score_

0.8802666666666666

In [40]:
pred = pipe_grid.best_estimator_.predict(X_test)
accuracy_score(y_test,pred)

0.88112

In [41]:
joblib.dump(pipe_grid, 'model/imdb_pipe.pkl')

['model/imdb_pipe.pkl']